In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
mlflow.set_tracking_uri("./mlruns")

In [4]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
C_values = [0.1, 1.0, 10.0]

for C in C_values:
    with mlflow.start_run():
        penalty = 'l2'
        solver = 'lbfgs'
        
        # Log parameters
        mlflow.log_param("penalty", penalty)
        mlflow.log_param("C", C)
        mlflow.log_param("solver", solver)
        
        # Train model
        model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=200, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluate model
        predictions = model.predict(X_test)
        acc = accuracy_score(y_test, predictions)
        mlflow.log_metric("accuracy", acc)
        print(f"Run with C={C}: accuracy = {acc:.4f}")
        
        # Log the model
        mlflow.sklearn.log_model(model, "model")

Run with C=0.1: accuracy = 0.9556


2025/02/04 18:22:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run with C=1.0: accuracy = 1.0000


2025/02/04 18:22:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run with C=10.0: accuracy = 1.0000


2025/02/04 18:22:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = "0"

runs = client.search_runs(experiment_ids=[experiment_id], order_by=["start_time DESC"])
print("Comparing runs:")
for run in runs:
    run_id = run.info.run_id
    params = run.data.params
    metrics = run.data.metrics
    print(f"Run ID: {run_id}")
    print(f"  Parameters: {params}")
    print(f"  Accuracy: {metrics.get('accuracy', 'N/A')}")
    print("-" * 30)


Comparing runs:
Run ID: 0c913ec25772498ba225504729031689
  Parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': '10.0'}
  Accuracy: 1.0
------------------------------
Run ID: a6addebbb6844785a32921e5a022255e
  Parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': '1.0'}
  Accuracy: 1.0
------------------------------
Run ID: 5dfb98959406485498a7520f9f1f62a6
  Parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': '0.1'}
  Accuracy: 0.9555555555555556
------------------------------
Run ID: d3f63e0eb71f4e51a73299a6e5c42b10
  Parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': '1.0'}
  Accuracy: 1.0
------------------------------
Run ID: cb2350471a2f43ceb9fd12c38609571f
  Parameters: {'plot_type': 'sine_wave'}
  Accuracy: N/A
------------------------------
Run ID: e1938591fdb84661bd5b6c33cd0b6321
  Parameters: {'learning_rate': '0.01'}
  Accuracy: 0.95
------------------------------
Run ID: c8bc5e56769b417694187b9661521adc
  Parameters: {'learning_rate': '0.01'}
  Accuracy: 0.95
--------